# Baseline Elo Training

This notebook:
1. Loads WHL 2025 CSV data and aggregates shifts → games (sum for goals)
2. Runs grid search over K-factor (classic Elo only—no home advantage, MOV, etc.)
3. Evaluates with combined RMSE, win accuracy—comparable to other models
4. Generates Round 1 predictions from `WHSDSC_Rnd1_matchups.xlsx`
5. Saves outputs to `output/predictions/baseline_elo/` and `output/models/baseline_elo/`

**Baseline Elo** = minimal Elo (chess-style) for simple benchmarking.

In [ ]:
import pandas as pd
import numpy as np
from itertools import product
import matplotlib.pyplot as plt
import json, os, sys, pathlib

# ---------- resolve python/ directory & set CWD ----------
_cwd = pathlib.Path(os.path.abspath('')).resolve()
if (_cwd / 'python').is_dir():
    _python_dir = _cwd / 'python'
elif _cwd.name == 'baseline_elo' and (_cwd.parent.parent / 'data').is_dir():
    _python_dir = _cwd.parent.parent
elif _cwd.name == 'training' and (_cwd.parent / 'data').is_dir():
    _python_dir = _cwd.parent
elif (_cwd / 'data').is_dir():
    _python_dir = _cwd
else:
    raise RuntimeError(f'Cannot locate python/ directory from {_cwd}')

os.chdir(_python_dir)
sys.path.insert(0, str(_python_dir))

from utils.baseline_elo import BaselineEloModel

plt.rcParams['figure.figsize'] = (12, 6)
print(f'CWD: {os.getcwd()}')

## Load CSV Data & Aggregate to Games

In [ ]:
# ── Load raw shift-level CSV and aggregate to game level ──
raw = pd.read_csv('data/whl_2025.csv')
print(f'Raw: {len(raw)} shift rows')

games_df = raw.groupby('game_id').agg(
    home_team  = ('home_team', 'first'),
    away_team  = ('away_team', 'first'),
    home_goals = ('home_goals', 'sum'),
    away_goals = ('away_goals', 'sum'),
    went_ot    = ('went_ot', 'max'),
).reset_index()

# Sort by game_id (chronological)
games_df['game_num'] = games_df['game_id'].astype(str).str.extract(r'(\\d+)').astype(int)
games_df = games_df.sort_values('game_num').reset_index(drop=True)

print(f'Games: {len(games_df)}')
print(f'Teams: {games_df["home_team"].nunique()}')
print(f'Avg home goals: {games_df["home_goals"].mean():.2f}, Avg away goals: {games_df["away_goals"].mean():.2f}')
print(f'Home win rate: {(games_df["home_goals"] > games_df["away_goals"]).mean():.1%}')
games_df.head()

## Train/Test Split & Grid Search (K-factor)

In [ ]:
# 80/20 chronological split (same as Elo/baseline)
split_idx = int(len(games_df) * 0.8)
train_df = games_df.iloc[:split_idx].copy()
test_df  = games_df.iloc[split_idx:].copy()

print(f'Train: {len(train_df)} games  |  Test: {len(test_df)} games')

# Baseline Elo has only k_factor and initial_rating
param_grid = dict(
    k_factor       = [16, 24, 32, 40, 48],
    initial_rating = [1200, 1500],
)
keys = list(param_grid.keys())
combos = list(product(*[param_grid[k] for k in keys]))
print(f'Grid: {len(combos)} configurations')

In [ ]:
# ── Grid Search ──
results = []

for i, vals in enumerate(combos):
    params = dict(zip(keys, vals))
    label = f"BaselineElo(k={params['k_factor']},init={params['initial_rating']})"
    try:
        model = BaselineEloModel(params)
        model.fit(train_df)
        metrics = model.evaluate(test_df)
        results.append({'config': label, **params, **metrics})
    except Exception as e:
        print(f'  FAILED {label}: {e}')

results_df = pd.DataFrame(results).sort_values('combined_rmse')
print(f'\n✅ {len(results_df)} configs evaluated')
print(f'Best combined RMSE: {results_df["combined_rmse"].min():.4f}')
print(f'Win accuracy range: {results_df["win_accuracy"].min():.1%} – {results_df["win_accuracy"].max():.1%}')
results_df.head(10)[['config','home_rmse','away_rmse','combined_rmse','win_accuracy']]

## Train Final Model & Rankings

In [ ]:
# ── Train on ALL data with best params ──
best_row = results_df.iloc[0]
best_params = {
    'k_factor': int(best_row['k_factor']),
    'initial_rating': int(best_row['initial_rating']),
}

final_model = BaselineEloModel(best_params)
final_model.fit(games_df)

# Evaluate on test split for reporting
eval_model = BaselineEloModel(best_params)
eval_model.fit(train_df)
final_metrics = eval_model.evaluate(test_df)

print('Final Baseline Elo model trained on all', len(games_df), 'games')
print(f'\nTest-set metrics:')
print(f'  Home RMSE:     {final_metrics["home_rmse"]:.4f}')
print(f'  Away RMSE:     {final_metrics["away_rmse"]:.4f}')
print(f'  Combined RMSE: {final_metrics["combined_rmse"]:.4f}')
print(f'  Win Accuracy:  {final_metrics["win_accuracy"]:.1%}')

print(f'\n🏆 Team Rankings (top 10):')
for rank, (team, rating) in enumerate(final_model.get_rankings(10), 1):
    print(f'  {rank:2d}. {team:20s}  {rating:.1f}')

In [ ]:
# ── Rating distribution visualization ──
rankings = final_model.get_rankings()
teams_r, ratings_r = zip(*rankings)

fig, ax = plt.subplots(figsize=(12, 6))
colors = ['#2ecc71' if r > 1500 else '#e74c3c' if r < 1500 else '#95a5a6' for r in ratings_r]
ax.barh(range(len(teams_r)), ratings_r, color=colors, edgecolor='black', linewidth=0.3)
ax.set_yticks(range(len(teams_r)))
ax.set_yticklabels(teams_r, fontsize=9)
ax.axvline(1500, color='gray', ls='--', lw=1, label='Initial (1500)')
ax.set_xlabel('Elo Rating')
ax.set_title('Baseline Elo Ratings — All Teams')
ax.legend()
ax.invert_yaxis()
plt.tight_layout()
plt.show()

## Round 1 Predictions

In [ ]:
# ── Load Round 1 matchups and predict ──
matchups = pd.read_excel('data/WHSDSC_Rnd1_matchups.xlsx')
print(f'{len(matchups)} matchups loaded')

home_col = [c for c in matchups.columns if 'home' in c.lower()][0]
away_col = [c for c in matchups.columns if 'away' in c.lower()][0]

preds = []
for i, row in matchups.iterrows():
    game = {'home_team': row[home_col], 'away_team': row[away_col]}
    h_goals, a_goals = final_model.predict_goals(game)
    winner, conf = final_model.predict_winner(game)
    preds.append({
        'game': i + 1,
        'home_team': row[home_col],
        'away_team': row[away_col],
        'pred_home_goals': round(h_goals, 2),
        'pred_away_goals': round(a_goals, 2),
        'predicted_winner': winner,
        'confidence': round(conf, 4),
    })

pred_df = pd.DataFrame(preds)
print(f'\nRound 1 Baseline Elo Predictions:')
print(pred_df.to_string(index=False))
print(f'\nAvg confidence: {pred_df["confidence"].mean():.1%}')

## Save Outputs

In [ ]:
os.makedirs('output/predictions/baseline_elo', exist_ok=True)
os.makedirs('output/models/baseline_elo', exist_ok=True)

# 1. Grid search comparison CSV
results_df.to_csv('output/predictions/baseline_elo/baseline_elo_comparison.csv', index=False)
print('✅ output/predictions/baseline_elo/baseline_elo_comparison.csv')

# 2. Round 1 predictions CSV
pred_df.to_csv('output/predictions/baseline_elo/round1_baseline_elo_predictions.csv', index=False)
print('✅ output/predictions/baseline_elo/round1_baseline_elo_predictions.csv')

# 3. Pipeline summary JSON
summary = {
    'model': 'BaselineElo',
    'best_params': best_params,
    'test_metrics': {k: round(v, 6) if isinstance(v, float) else v for k, v in final_metrics.items()},
    'team_rankings': {team: round(r, 1) for team, r in final_model.get_rankings()},
    'predictions': preds,
}
with open('output/predictions/baseline_elo/baseline_elo_pipeline_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)
print('✅ output/predictions/baseline_elo/baseline_elo_pipeline_summary.json')

print(f'\n🏆 SUMMARY: Combined RMSE {final_metrics["combined_rmse"]:.4f}, Win Acc {final_metrics["win_accuracy"]:.1%}')